In [1]:
fulldir="/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/"
name="ppl2/"

In [2]:
import shutil as sh
import cv2
import os
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm
import os.path

In [3]:
# file = open(fulldir+"pixels.txt","a+")

In [4]:
image = fulldir+name
ext = ".bmp"
key = "abcdefghijklm"
i=0

In [5]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0], image_size[1],color

In [6]:
def LogisticEncryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    L = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1
    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    C = round((L*L_y*10**4) % 256)
    C_r = round((L*L_y*10**4) % 256)
    C_g = round((L*L_y*10**4) % 256)
    C_b = round((L*L_y*10**4) % 256)
    x = 4*(S_x)*(1-S_x)
    y = 4*(S_y)*(1-S_y)
    imageMatrix,dw, dl, color = getImageMatrix(imageName)
    LogisticEncryptionIm = []
    for i in range(dw):
        row = []
        for j in range(dl):
            i_x=0
            while x <0.8 and x > 0.2:
                x = 4*x*(1-x)
                i_x=i_x+1
                if i_x>=30:
                    print("i_x")
                    return -1
            i_y=0
            while y <0.8 and y > 0.2:
                y = 4*y*(1-y)
                i_y=i_y+1
                if i_y>=30:
                    print("i_y")
                    return -1
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                C_r =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][0]) % N) ^ ((C_r + key_list[7]) % N)
                C_g =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][1]) % N) ^ ((C_g + key_list[7]) % N)
                C_b =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][2]) % N) ^ ((C_b + key_list[7]) % N)
                row.append((C_r,C_g,C_b))
                C = C_r
            else:
                C = ((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j]) % N) ^ ((C + key_list[7]) % N)
                row.append(C)
            x = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            y = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        LogisticEncryptionIm.append(row)
    im = Image.new('L', (dw, dl))
    if color:
        im = Image.new('RGB', (dw, dl))
    else: 
        im = Image.new('L', (dw, dl)) # L is for Black and white pixels    
    pix = im.load()
    for x in range(dw):
        for y in range(dl):
            pix[x, y] = LogisticEncryptionIm[x][y]
    return im
    #im.save(imageName[0:-8]+"encryp"+ext, "BMP")

In [7]:
vals=[]

In [8]:
def EncryptionResult(img):
    cnt=0
    face = []
    modelFile = fulldir+"res10_300x300_ssd_iter_140000.caffemodel"
    configFile = fulldir+"deploy.prototxt.txt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (800, 800)), 1.0,(800, 800), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    #to draw faces on image
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
#           print(x,y,x1,y1)
            if y1<0: y1=0
            if x1<0: x1=0
            if x<0: x=0
            if y<0: y=0
            if y1>=h: y1=h-1
            if x1>=w: x1=w-1
            if x>=w: x=w-1
            if y>=h: y=h-1
            if x1-x>0 and y1-y>0:
                face.append((x, y, x1, y1))
                cv2.rectangle(img, (x, y), (x1-1, y1-1), (255, 0, 0), 1)
    count=0
    temp=0
    for (x, y, x1, y1) in face:
        print((x,y,x1,y1))
        #if x>x1 and y>y1:
        cropped_image = img[y+1:y1-1, x+1:x1-1]
        #elif x<x1 and y<y1:
         #   cropped_image = img[y1+1:y-1, x1+1:x-1]
       # else:
        #    continue
        filename=fulldir+name+"temp"+ext
        #print(filename)
        #print(cropped_image)
        cv2.imwrite(filename, cropped_image)
        imgt = LogisticEncryption(filename, key)
        if imgt==-1:
            print("infinite loop")
            return -1
        temp+=1
        LogisticResult=cv2.cvtColor(np.array(imgt),cv2.COLOR_RGB2BGR)
        img[y+1:y1-1, x+1:x1-1] = LogisticResult
        #cv2.imwrite(fulldir+name+'res.png', LogisticResult)
        count=count+1
    return 0

In [9]:
# Read the video from specified path
cam = cv2.VideoCapture(fulldir+name+name[:-1]+".mp4")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)

30.0


In [10]:
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))

print(height, width)

1080 1920


In [11]:
currentframe = 0
video_name = 'encryptedvideo.avi'
os.chdir(fulldir+name+"results")
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'FFV1'), fps, (width, height))

while(True):
    ret,Img = cam.read()
    if ret:
        print("frame:",currentframe)
        if currentframe<10:
            cv2.imwrite(fulldir+name+"frames/00"+str(currentframe)+".png", Img)
        elif currentframe<100:
            cv2.imwrite(fulldir+name+"frames/0"+str(currentframe)+".png", Img)
        else:
            cv2.imwrite(fulldir+name+"frames/"+str(currentframe)+".png", Img)
        frameout=EncryptionResult(Img)
        #print(frameout)
        if frameout==-1:
            currentframe += 1
            continue
        if currentframe<10:
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult00"+str(currentframe)+".bmp", Img)
        elif currentframe<100:
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult0"+str(currentframe)+".bmp", Img)
        else:
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult"+str(currentframe)+".bmp", Img)
        video.write(Img)
        currentframe += 1
    else:
        break

frame: 0
(337, 152, 493, 362)
(873, 215, 1018, 408)
(1374, 173, 1520, 374)
frame: 1
(337, 152, 493, 362)
(873, 215, 1018, 409)
(1374, 173, 1520, 374)
frame: 2
(336, 152, 493, 362)
(872, 215, 1018, 409)
(1374, 172, 1520, 374)
frame: 3
(336, 152, 493, 362)
(873, 215, 1019, 409)
(1374, 172, 1520, 374)
frame: 4
(336, 152, 493, 362)
(873, 215, 1018, 409)
(1374, 173, 1519, 374)
frame: 5
(336, 152, 493, 362)
(873, 214, 1018, 409)
(1374, 173, 1519, 374)
frame: 6
(336, 152, 493, 362)
(873, 214, 1020, 410)
(1375, 173, 1519, 374)
frame: 7
(336, 152, 493, 362)
(874, 214, 1020, 410)
(1374, 173, 1519, 374)
frame: 8
(336, 152, 493, 362)
(875, 215, 1020, 410)
(1374, 173, 1519, 374)
frame: 9
(336, 152, 493, 362)
(876, 215, 1020, 410)
(1374, 173, 1519, 374)
frame: 10
(336, 152, 493, 362)
(876, 215, 1020, 410)
(1374, 173, 1519, 374)
frame: 11
(336, 152, 493, 362)
(877, 214, 1021, 410)
(1374, 173, 1519, 374)
frame: 12
(336, 152, 493, 362)
(880, 213, 1022, 410)
(1374, 173, 1519, 374)
frame: 13
(336, 152, 4

In [12]:
cv2.destroyAllWindows()
video.release()